
Projet de construction d'un modèle qui tentera de prédire si quelqun remboursera ou non son prêt en se basant sur des informations historiques

In [ ]:
# Import des librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Import du jeu de données
df=pd.read_csv('lending_club_loan_two.csv')

In [ ]:
df.info()

# Analyse Exploratoire

In [ ]:
# Création d'un graphique de comptage pour la variable cible
sns.countplot(x='loan_status',data=df)

In [ ]:
# création d'un histogramme pour les varibales continues. exemple pour voir la distribution des montants accordés
plt.figure(figsize=(12,4))
sns.distplot(df['loan_amnt'],kde=False,bins=40)
plt.xlim(0,45000.0)

In [ ]:
# calcul de la corrélation entre toutes les variables continues en utilisant la méthode .corr()
#df.corr()
# Visualisation avec une heatmap
plt.figure(figsize=(12,7))
sns.heatmap(df.corr(),annot=True,cmap='viridis')
plt.ylim(10,0)

In [ ]:
# Constat d'une relation presque parfaite avec la feature 'installment'
# Exploration de cette feature en effectuant un nuage de point et voir si cette relation a du sens avec la variable à prédire
# on veut s'assurer ne pas avoir de fuite de données avec cette feature dans notre label
# on doit toujours être sur qu'il n'y a pas 1 seul feature qui serait un parfait prédicteur du label à prédire car cela signifirait qu'il ne s'agit pas
# d'une simple feature mais probablement d'information dupliqué qui serait très similaire au label
plt.figure(figsize=(12,8))
sns.scatterplot(x='installment',y='loan_amnt',data=df)

In [ ]:
# Création d'un box plot montrant la relation entre l'état du prêt (loan_status) et le montant du prêt
# permet de répondre si il y a une relation entre les emprunt très elevé VS enprunt peu élevé et le fait qu'il soit totalement soldé
sns.boxplot(x='loan_status',y='loan_amnt',data=df)
# En moyenne la boîte charged off(emprunts non soldés) est légèrement plus haute  ce qui signifie que lorsque le montant de prêt est élevé
# on a plus de chance que l'emprunt ne soit pas remboursé, logique car plis difficile de rembourser des montants élevés que des plus petits montants

In [ ]:
# Calucl de stats sommaires pour les montants du prêt regroupés  par la feature loan status
df.groupby('loan_status')['loan_amnt'].describe()

In [ ]:
# Exploration des colonnes Grade et subGrade que lending_club attribue aux prêts: voir les notations et sous notations possibles
#df['grade'].unique()
df['sub_grade'].unique()
# on constate que les sous notations contiennent les infos des notations réelles

In [ ]:
# création d'un graphique de comptage par notation (grade) et en définissant 'loan_status' avec le paramètre 'hue'
plt.figure(figsize=(12,7))
sns.countplot(x='grade',data=df,hue='loan_status')
# on peut voir qu'i ya une différenciation entre les emprunts totalement rembournés et les non remboursés selon la notation du crédit 
# on peut voir le pourcentage d'emprunt non remboursé augmente à mesure que les lettres augmentent
# A la lettre G on constate que 50% d'emprunt sont remboursés et 50% d'emprunts sont non remboursés

In [ ]:
# Affichage d'un graphique de comptage par sous_notation (sub_grade) de qualité de prêt
# il s'agit de la distribution à travers tous le dataset : combiende A1 on dispose et ....
# on constate majoritairement que les crédits sont de notations A,B,C,D--> représentant les crédits les moins risqués qui ont moins de chance pas être remboursé
plt.figure(figsize=(12,4))# pour redimensionner le graphique
subgrade_order=sorted(df['sub_grade'].unique())#  triage sur les sous notations avec l'appel de la fonction 'sorted'
sns.countplot(x='sub_grade',data=df,order=subgrade_order, palette='coolwarm')
# Les meilleures notations sont en bleu et les pires sont en rouges 

In [ ]:
# graph qui nous permet de comparer par sous notation la ration compètement remboursé VS non Remboursé
plt.figure(figsize=(12,4))# pour redimensionner le graphique
subgrade_order=sorted(df['sub_grade'].unique())#  triage sur les sous notations avec l'appel de la fonction 'sorted'
sns.countplot(x='sub_grade',data=df,order=subgrade_order, palette='coolwarm',hue='loan_status')
# pour les pires notations le taux de fully_paid et charged off sont presque les mêmes 
# ce qui semble pertinent d'investiguer si c'est pertinent d'accorder des prêts au client ayant ces notations F ou G

In [ ]:
# Il semble que F et G ne sont pas remboursés très souvent: création d'un graph de comptage en isolant nos deux notations
f_and_g=df[(df['grade']== 'G') |(df['grade']=='F')]# création d'un sous dataframe en filtrant sur nos deux notations
plt.figure(figsize=(12,4))# pour redimensionner le graphique
subgrade_order=sorted(f_and_g['sub_grade'].unique())#  
sns.countplot(x='sub_grade',data=f_and_g,order=subgrade_order, palette='coolwarm',hue='loan_status')# utilisation du dataframe filtré
# si le client est noté à G5 il est presqu'aussi probable qu'il rembouse totalement l'emprunt et qu'il ne rembourse pas

In [ ]:
# Création d'une nouvelle colonne  'loan_repaid' qui contiendra 1 si le statut du prêt était fully_paid et o si charged off
# qui sera notre label à prédire
df['loan_repaid']= df['loan_status'].map({'Fully Paid':1,'Charged Off':0})

In [ ]:
df[['loan_repaid','loan_status']]

In [ ]:
# création d'un diagramme en barre (bar plot) entre les features numériques et la nouvelle colonne et la nouvelle colonne laon_repaid qui est notre label

df.corr()['loan_repaid'].sort_values()[:-1].plot(kind='bar')
df.corr()['loan_repaid'].sort_values().drop('loan_repaid').plot(kind='bar')
# Pour quelle feature numérique à la plus grande corrélation avec notre label
# int_rate est négativement fortmement corrélé : si le taux d'int est élevé on a plus de difficulté à rembourser l'emprunt

# Traitement des données manquantes

In [ ]:
# convertir la série en pourcentage du totale du data frame
#df.isnull().sum()
df.isnull().sum()/len(df)*100

In [ ]:
# Features emp_title et emp_length voir si il est possible de les supprimer 
# Combien y a t-il d'emploi unique?
df['emp_title'].nunique()
# on obtient une tonne de métier différent parmis les emprunteurs

In [ ]:
df['emp_title'].value_counts()
# Il y a trop de titres de postes uniques pour essayer de les convertir en une feature de vraibles dummy. on va donc supprimer cette colone
# comme solution(Feature_engineering), on peut catégoriser ces métiers comme métiers à haut revenu, revenus moyen et faible revenu
# il faut mapper 37076 titre de job différents

In [ ]:
# suppression de la colonne emp_title
df=df.drop('emp_title',axis=1)

In [ ]:
# création d'un graphique de comptage de la colonne emp_length en faisant un tri
sorted(df['emp_length'].dropna().unique())

In [ ]:
emp_length_order=['< 1 year','1 year',
 '2 years',
 '3 years',
 '4 years',
 '5 years',
 '6 years',
 '7 years',
 '8 years',
 '9 years',
 '10+ years']

In [ ]:
plt.figure(figsize=(12,4))
sns.countplot(x='emp_length',data=df,order=emp_length_order)

In [ ]:
# Tracer un décompte avec un paramètre hue de séparation entre Fully Paid et Charged Off
# l'état de l'emrunt si soldé ou non
# relation entre Fully Paid et charged Off par durée d 'emploi
plt.figure(figsize=(12,4))
sns.countplot(x='emp_length',data=df,order=emp_length_order,hue='loan_status')

In [ ]:
# on veut savoir le pourcentage de personnes par catégories d'emploi qui n'ont pas remboursé leur prêt
emp_co=df[df['loan_status']=='Charged Off'].groupby('emp_length').count()['loan_status'] # selection des personnes qui  n'ont pas remboursé et on les groupe par durée d'emploi puis on les compt

In [ ]:
emp_fp=df[df['loan_status']=='Fully Paid'].groupby('emp_length').count()['loan_status']

In [ ]:
# Obtention du rato entre les deux séries
# pourcentage des personnes qui sont soldés leur emprunt VS ceux qui n'ont pas soldés
emp_len=emp_co/(emp_fp+ emp_co)

In [ ]:
emp_len.plot(kind='bar')
# Différence pas assez significative pour conserver cette feature
# du fait de la siilarité des résultats quelque soit la catéforie durée d'emploi on peut supprimer cette feature

In [ ]:
df=df.drop('emp_length',axis=1)

In [ ]:
# Examinons le titre du prêt Vs colonne purpose
df['purpose'].head()
#df['title'].head()
# suppression de la colonne title car les infos fournis sont identiques à purpose

In [ ]:
df=df.drop('title',axis=1)

In [ ]:
# Variable mort_acc: création d'un graph de contage
df['mort_acc'].value_counts()
# la majorité des personnes ont o autres compte d'hypothèques représentant 25% des données
# Approche pour traiter les données manquantes c'est de découvrir une feature qui a toute l'information qui aurait une très forte corrélation avec mort_acc
# voir ensuite si on peut l'utiliser pour compléter 'info manquante

In [ ]:
# détermination de la colonne qui fortmement corrélé avec mort_acc
df.corr()['mort_acc'].sort_values
# total_acc semble avoir une bonne corrélation positive

In [ ]:
# regroupement du dataframe par total_acc et calculon la valeur moyenne de mort_acc par entrée de total_acc en utilisant la méthode fillna()
total_acc_avg=df.groupby('total_acc').mean()['mort_acc']
# pour le remplacement des valeurs manquantes


In [ ]:
# construction d'une fonction pour remplir les valeurs manquantes d'un data frame avec deux colonnes 
def fill_mort_acc(total_acc,mort_acc):
  if np.isnan(mort_acc):
    return total_acc_avg['total_acc']
  else:
    return mort_acc

In [ ]:
# Suppression des colones revo_util et pub_reck_bankcies représentent moins de 0.5% du dataset en utilisant la méthode dropna()
df=df.dropna()

# Traitement des données catégoriques

In [ ]:
#Lister toutes les colonnes catégoriques: utilisation de la méthode d_types
df.select_dtypes(['object']).columns

In [ ]:
# feature term
df['term'].value_counts()
# colone binaire soit 36 mois soit 60 mois 
# transformation en colonne numérique

In [ ]:
df['term']=df['term'].apply(lambda term: int(term[:3]))

In [ ]:
df['term'].value_counts()

In [ ]:
# feauture grade
df['grade'].value_counts()


In [ ]:
# suppression de la feature grade car toutes les infos sont dans la feature sub_grade
df=df.drop('grade',axis=1)

In [ ]:
# conversion de subgrades en variables dummies puis on va les concatener au datframe d'origine
# puis supprimer subgrade d'origine et d'ajouter drop_first=True à l'appel get_dummies() pour empêcher le piège des varaibles multiples
# pour éviter l'encodage d'info dupliquées
# Obtenir les varaibles dummies
subgrade_dummies=pd.get_dummies(df['sub_grade'],drop_first=True)# ce qui évite l'encodage d'info dupliquée ainsi on obtient k-1 Dummies


In [ ]:
#pour les 4 features
dummies=pd.get_dummies(df[['verification_status','initial_list_status','purpose','application_type']],drop_first=True)# transformation en variable dummies
df=df.drop(['verification_status','initial_list_status','purpose','application_type'],axis=1)
df=pd.concat([df,dummies],axis=1)

In [ ]:
#feature home_ownership
# evaluation des valeurs de cette feature
df['home_ownership'].value_counts()

In [ ]:
# on constate qu'il y a peu de personne dans Any et none , on va les mettre dans other
# remplacer none et any par others en utlisant  ma méthode replace 
df['home_ownership']= df['home_ownership'].replace(['NONE','ANY'],'OTHER')

In [ ]:
# Conversion en variable dummies
dummies=pd.get_dummies(df['home_ownership'],drop_first=True)# transformation en variable dummies
df=df.drop('home_ownership',axis=1)
df=pd.concat([df,dummies],axis=1)

In [ ]:
df=pd.concat([df,dummies],axis=1)

In [ ]:
# feature adress
df['zip_code']=df['address'].apply(lambda adress: adress[-5:])# application de la fonction pour récupérer les 5 derniers chifress
# il faut extraire le code postal ou le zip code depuis l'adresse

In [ ]:
# conversion de cette colone zip_code en dummies
# création d'une catégorie par zip_code
df['zip_code'].value_counts()

In [ ]:
dummies=pd.get_dummies(df['zip_code'],drop_first=True)# transformation en variable dummies
df=df.drop('zip_code',axis=1)
df=pd.concat([df,dummies],axis=1)

In [ ]:
# suppression de la colone adresse
df=df.drop('address',axis=1)

In [ ]:
# feature issue: on sait pas à lavance si un prêt sera émis ou pas en utilisant notre modèle 
# on peut l'éliminer 
df=df.drop('issue_d',axis=1)

In [ ]:
# Feature earliest_cr_line
# extraction de l'année depuis cette feature
df['earliest_cr_line']

In [ ]:
df['earliest_cr_line']= df['earliest_cr_line'].apply(lambda date: int(date[-4:]))

In [ ]:
df['earliest_cr_line']

# Repartition entre données d'entraînement et de test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Suppression de la colonne loan_status créée précédement car c'est une info dupliquée de la colonne loan_repaid
df=df.drop('loan_status',axis=1)

In [ ]:
# X: pour les features
#y: pour le label à prédir c'est à dire loan_repaid
X=df.drop('loan_repaid',axis=1).values# pour avoir juste les valeurs et satisfaire tensor flow
y=df['loan_repaid'].values

In [ ]:

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=101)

In [ ]:
# Normalisation des données

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train=scaler.fit_transform(X_train)# Application de la méthode fit_transform uniquement sur les données d'entraînement
X_test=scaler.transform(X_test) # pour les données test on n'adapte pas on les transorme seulement pour éviter toute fuite de données

In [ ]:
# Création du modèle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [ ]:
X_train.shape
# il semble avoir 81 features et on va faire correspondre ce nombre à la première couche
# la première couche doit correspondre au nombre de features

In [ ]:
model=Sequential()

model.add(Dense(units=81,activation='relu'))# Première couche (couche d'entrée/input layer)
model.add(Dropout(0.2))

model.add(Dense(units=41,activation='relu'))# 1ère couche cacheé
model.add(Dropout(0.3))

model.add(Dense(units=40,activation='relu'))# 2ème couche cacheé
model.add(Dropout(0.3))

model.add(Dense(units=1,activation='sigmoid'))# couche de sortie (ouptut layer)

model.compile(loss='binary_crossentropy',optimizer='adam')

- Utilisation des call backs pour les arrêts anticipés de l'entrâinement  dans le but contrôler le surapprentissage
- Ce qui permet de determiner le bon nombre d'epochs 
- montiror: contrôler la perte de validation
- min_delta: changement minimum requis
- patience: nombre d'epochs avec aucune amélioration avec le quel l'entrainement s'arrête
- mode: c'est ce qu'on éssaie de faire: soit min: minimiser les éléments monitorés ex(la loss)
- si notre métrique est une accuracy par exemple: on devrait maximiser

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=25)

In [ ]:
model.fit(X_train,y_train,epochs=100,batch_size=256,validation_data=(X_test,y_test),callbacks=early_stop)

In [ ]:
# Tracer la perte
losses=pd.DataFrame(model.history.history)

In [ ]:
# Prediction
# comme il s'agit d'un classification binaire on ajoute >0.5 et convertir en entier 0et 1 avec astype
from sklearn.metrics import classification_report,confusion_matrix
y_pred=(model.predict(X_test)>0.5).astype('int32')

In [ ]:
# rapport de classification
print(classification_report(y_test,y_pred))

In [ ]:
# comparaison avec le dataframe d'origine
df['loan_repaid'].value_counts()

In [ ]:
285936/len(df)


In [ ]:


import random
random.seed(102)
random_ind= random.randint(0,len(df))# création d'un index aléatoire et à partir de cet index on prend un client et on lui  retire la valeur du label et on affiche
# ces features

new_client= df.drop('loan_repaid',axis=1).iloc[random_ind]
new_client

In [ ]:
new_client=scaler.transform(new_client.values.reshape(1,81))

In [ ]:
(model.predict(new_client)>0.5).astype('int32')
# le modèl prédit la classe 1 donc il finira par rembourser son prêt

In [ ]:
# vérifions si la personne a vraiment rembourser: on va vérifier la valeur  de la colonne loan_repaid qui est le label à prédire 
df.iloc[random_ind]['loan_repaid']